# Sumarization

In [2]:
import polars as pl

def view_string(long_string, chunk_size=100):     
    return [long_string[i:i+chunk_size] for i in range(0, len(long_string), chunk_size)]

In [3]:
df = pl.read_csv('/home/sebacastillo/willow/output/sample.csv')

In [4]:
df.columns

['date_extract', 'date_article', 'content', 'link', 'authors']

In [5]:
df

date_extract,date_article,content,link,authors
str,str,str,str,str
"""2023-08-09 19:…","""2023-08-09T00:…","""9 Ago, 2023 La…","""https://www.in…","""n-a"""
"""2023-08-09 19:…","""2023-08-09T10:…","""LOS POTRILLOS.…","""https://www.pr…","""Gabriela Lorei…"
"""2023-08-09 19:…","""2023-07-31T00:…","""Para lograr un…","""https://www.el…","""n-a"""


In [6]:
text = df['content'][0]

In [7]:
view_string(text)

['9 Ago, 2023 La fiscal Silvia Bussano, de la UFI N°7 de Lanús, no tiene aún en sus manos el informe p',
 'reliminar de la autopsia, pero sí los forenses que analizaron el cuerpo de Morena Domínguez le adela',
 'ntaron sus conclusiones: la nena de 11 murió a causa de una hemorragia interna producto de una grave',
 ' lesión en el hígado. Así se lo confirmaron a Infobae fuentes de la investigación. “La autopsia reve',
 'ló que la nena falleció por un desgarro en el hígado producto del golpe profundo que recibió”, expli',
 'caron sobre los resultados de la pericia que se realizó en la Morgue Judicial de Lomas de Zamora. O ',
 'sea, los asesinos la piña en la panza que le dieron a la nena fue mortal. Por el crimen de Morena ha',
 'y dos detenidos: se trata de los hermanos Darío Humberto Madariaga, alias Lolo y de 25 años; y Migue',
 'l Ángel Madariaga, de 28.  Ambos están acusados del delito de “homicidio en ocasión de robo en concu',
 'rso real con encubrimiento”. Es que, fuentes del caso

# Text Sumarization

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_str = "IIC/mt5-spanish-mlsum"
tokenizer = AutoTokenizer.from_pretrained(model_str)
model = AutoModelForSeq2SeqLM.from_pretrained(model_str)

In [9]:
input_ids = tokenizer(text, return_tensors="pt").input_ids
output_ids = model.generate(input_ids, max_new_tokens=200, penalty_alpha=0.6, top_k=4,)[0]
print(tokenizer.decode(output_ids, skip_special_tokens=True))

La autopsia revela que la nena de 11 murió a causa de una hemorragia interna. Los forenses adelantan que la investigación revela que la nena falleció a causa de una hemorragia interna producto de una grave lesión en el hígado


In [10]:
articles = df['content'].to_list()

In [24]:
articles_tokenized = []
articles_summaries = []
for article in articles:
    input_ids = tokenizer(article, return_tensors="pt").input_ids
    articles_tokenized.append(input_ids)
    output_ids = model.generate(input_ids, max_new_tokens=200, penalty_alpha=0.6, top_k=4,)[0]
    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    articles_summaries.append(summary)

In [25]:
articles_summaries

['La autopsia revela que la nena de 11 murió a causa de una hemorragia interna. Los forenses adelantan que la investigación revela que la nena falleció a causa de una hemorragia interna producto de una grave lesión en el hígado',
 'Los policías de Posadas se juzgan por la muerte de Mirta Carmen Rosa. El Tribunal Penal 2 de Posadas juzga a dos policías por varias acusaciones vinculadas a la muerte violenta de una docente',
 'El consejo para ahorrar energía y agua al utilizar el lavarropa. La Federación Argentina de Cooperativas de Electricidad y Otros Servicios Públicos (FACE) debe dejar una serie de consejos para alcanzar la eficiencia energética']

In [26]:
articles_tokenized

[tensor([[   774,  60850,    261, 107731,    501,  17182,    259,  93388,  94214,
            2031,    261,    269,    283,    581,  17350,    441,    755,    487,
             269,  13960,  19630,    261,    375,   2812,    259,    262,   4658,
             289,   1996,    674,    337,    362,  39995,    259,  92096,    269,
             283,   1864,  41871,    262,    261,   2757,   5910,    595,    332,
           47938,    319,  17360,   9694,    362,    317,  30420,    269,   3398,
             377,   5438,   4979, 117929,    340,    259, 101763,   9694,   1996,
             259,  49666,    299,    267,    283,  46145,    269,    730,  13878,
             553,    259,    262,   9034,    269,    573,  95359, 106690,    262,
           20855,  29139,    269,    573,  18582,    340,   9678,    289,    362,
             259,   7550,  27365,    260,   1477,    420,    303,    707,  56513,
            2756,    259,    262,  10402,    835,    265,    259,  51468,    263,
             269